In [ ]:
# test object categorization under noise as an RL task
import sys

sys.path.append('/home/squax/GitHub/CMS/')
from agent.reinforcement import *
from brain.monitor import *
from brain.models import ActorCriticModel
from brain.networks import *
from world.base import World
from world.data import *
from world.tasks import *
from brain.links import *
import matplotlib
import matplotlib.pyplot as plt
import time
import tools
%matplotlib inline 

start_time=time.strftime("%H:%M:%S")
# parameters
n_epochs = 1
n_level = [0.5]
coh_all=(100-np.logspace(np.log10(100),np.log10(3.125),num=6))/100#[0.5,0.75,0.85,0.92]# [1 - x for x in [0.08, 0.15, 0.25, 0.5]]

rwrds = [0, 1, 0] ###test -> 0
i_class = [0, 1]
train_batch = 50000
test_batch = 20000
fix_length=3
trial_length=50
gamma = 0.999

num_obs = np.zeros(len(coh_all))
bin_obs = np.zeros((len(coh_all), 100))
acc = np.zeros((len(coh_all),100))
acc1 = np.zeros(len(coh_all))

nobs_all = np.zeros((len(n_level), 40))
mobs_all = np.zeros((len(n_level), 40))

gpu=-1

np.random.seed(1)

if gpu==1:
    chainer.cuda.get_device_from_id(1).use()

for iii in xrange(len(n_level)):
    # get training data - note that we select a subset of datapoints (n_samples is nr of samples per class)
    train_data = MNISTData(test=False, convolutional=False, n_samples=1000, classes = [0,1],centre=False)

    # define iterator
    data_iter = DataTaskMnist(train_data, batch_size=1, n_batches=train_batch, coh_all=coh_all, rewards=rwrds, noise_method=1,
                         fix_length=fix_length, trial_length=trial_length)

    # an actor-critic model assumes that the predictor's output is number of actions plus one for the value
    n_output = data_iter.n_output
    n_input = data_iter.n_input
    # define brain of agent
 
    model = ActorCriticModel(RNN2(n_input, n_output, n_hidden=100, link=GRU_custom))  
                                                            ###test ->100
    # save model
    fname = 'result/model_mnist22_fl'+str(fix_length)+'_l' + str(trial_length) + '_g' + str(gamma).replace('.', '') + '_r' + str(rwrds[0]).replace('.', '')
    #fname = 'result/model_mnist32_fl'+str(fix_length)+'_l' + str(trial_length) + '_g' + str(gamma).replace('.', '') + '_r' + str(rwrds[0]).replace('.', '')

    ###test

    #_fl'+str(fix_length)
    model.load(fname)

    monitor = Oscilloscope(names=['p1', 'p2', 'p3'])
    model.add_monitor(monitor)
    # define agent: only entropy on state actions
    agent = ActorCriticAgent(model, chainer.optimizers.Adam(alpha=0.0005), cutoff=train_batch, beta=1e-2, gamma=gamma, aac=True, gpu=gpu)  # Adam(alpha=0.0005)

    # add gradient clipping
    agent.optimizer.add_hook(chainer.optimizer.GradientClipping(1))

    agent.add_monitor(monitor)
    # # add oscilloscope
    agent.add_monitor(Oscilloscope(names=['cumulative reward']))
    agent.add_monitor(Monitor(names=['action']))
    data_iter.add_monitor(Monitor(names=['state','step']))

    monitor = Oscilloscope(names=['accuracy','resp_ratio'])
    data_iter.add_monitor(monitor)
    agent.add_monitor(monitor)

    # define world
    world = World(agent)

    # create test data
    val_data = MNISTData(test=True, convolutional=False, n_samples=1000, classes=[0,1], centre=False)
    val_iter = DataTaskMnist(val_data, batch_size=1, n_batches=test_batch, coh_all=coh_all, rewards=rwrds, noise_method=1,
                        fix_length=fix_length, trial_length=trial_length)

    # add monitor to model and iterator
    agent.add_monitor(Monitor(names=['action']))
    val_iter.add_monitor(Monitor(names=['state','step']))
    val_iter.add_monitor(Monitor(names=['coh']))

    monitor = Oscilloscope(names=['accuracy'])
    val_iter.add_monitor(monitor)
    agent.add_monitor(monitor)

    monitor1 = Oscilloscope(names=['p1', 'p2', 'p3','hidden-1','hidden-2'])
    model.add_monitor(monitor1)


    monitor = Monitor()
    agent.add_monitor(monitor)
    val_iter.add_monitor(monitor)

    # run in test mode
    world.test(val_iter, n_epochs=1, plot=0)

In [ ]:
# get variables
Y1 = monitor['action']
T1 = monitor['state']
C1 = monitor['coh']

p1 = monitor1['p1']
p2 = monitor1['p2']
p3 = monitor1['p3']
h1 = monitor1['hidden-1']
h2 = monitor1['hidden-2']

nacc = np.zeros((len(coh_all),100))

A1 = np.zeros(Y1.shape)
C = np.zeros((2, len(Y1)))

temp = 0
for ii in xrange(len(Y1)):
    temp = temp + 1
    A1[ii] = temp
    if Y1[ii] != (data_iter.n_output - 1) or temp == trial_length+1:
        C[T1[ii], ii] = temp
        temp = 0

p1m = np.zeros((2, 40, 40))
p1m[:]=np.nan
p2m = np.zeros((2, 40, 40))
p3m = np.zeros((2, 40, 40))

for ic in xrange(2):
    for ii in xrange(40):
        idx = np.where(C[ic, :] == ii + 1)
        idx1 = []
        for i2 in xrange(ii + 1):
            idx1 = [x - ii + i2 for x in list(idx)]
            p1m[ic, ii, i2] = np.mean(p1[idx1])
            p2m[ic, ii, i2] = np.mean(p2[idx1])
            p3m[ic, ii, i2] = np.mean(p3[idx1])

# only focus on those trials at which a decision is being made
idx = np.where(Y1 != data_iter.n_output - 1)[0]
Y = Y1[idx]
T = T1[idx]
A = A1[idx]
C2 = C1[idx]

idx = np.where(A > fix_length)[0]
Y3 = Y[idx]
T3 = T[idx]
A3 = A[idx]
C3 = C2[idx]

B = np.equal(Y, T)

stepsize=2
bins=50/stepsize
bin_obs = np.zeros((len(coh_all), bins))
bin_obs_co = np.zeros((len(coh_all), bins))
bin_obs_uc = np.zeros((len(coh_all), bins))
print(max(A))
coh=np.unique(C2)
for ii in xrange(len(coh)):
    idx1 = list(np.where(C2 == coh[ii]))
    num_obs[ii] = np.mean(A[idx1])
    bin_obs[ii] = np.histogram(A[idx1].astype('int32'), bins=bins,range=(1,trial_length+1))[0]
    
    idx11 = list(np.where((C2 == coh[ii])&(B==1)))
    idx12 = list(np.where((C2 == coh[ii])&(B==0)))

    bin_obs_co[ii] = np.histogram(A[idx11].astype('int32'), bins=bins,range=(1,trial_length+1))[0]
    bin_obs_uc[ii] = np.histogram(A[idx12].astype('int32'), bins=bins,range=(1,trial_length+1))[0]

### accuracy

for ii in xrange(len(coh)):
    idx1 = list(np.where(C2 == coh[ii]))
    acc1[ii] = np.sum(B[idx1]) / float(len(idx1[0]))
    for iii in xrange(50):
        idx2 = list(np.where((A == iii + 1) & (C2 == coh[ii])))
        if len(idx2[0])>1:
            acc[ii,iii] = np.sum(B[idx2])
            nacc[ii,iii] = float(len(idx2[0]))

nobs_acc=np.zeros((len(coh),2))
acc_nobs=np.zeros((len(coh),2))   

nobs_acc_err=np.zeros((len(coh),2))
P=np.zeros((len(coh)))
P2=np.zeros((len(coh)))

Corr=np.zeros((len(coh)))
for ii in xrange(len(coh)):
    idx1 = list(np.where((C2 == coh[ii]) & (B==1)))
    idx2 = list(np.where((C2== coh[ii]) & (B!=1)))
    idx3 = list(np.where(C2== coh[ii]))
    idx4 = np.argsort(A[idx3])
    B2= B[idx3]
    acc_nobs[ii,1]=np.mean(B2[idx4[:(len(idx3[0])/2)]])
    acc_nobs[ii,0]=np.mean(B2[idx4[(len(idx3[0])/2):]])
    nobs_acc[ii,1]=np.mean(A[idx1])
    nobs_acc[ii,0]=np.mean(A[idx2])
    nobs_acc_err[ii,1]=sst.sem(A[idx1])
    nobs_acc_err[ii,0]=sst.sem(A[idx2])
    
    if0=np.sum(B2[idx4[:(len(idx3[0])/2)]]==False)
    if1=np.sum(B2[idx4[:(len(idx3[0])/2)]]==True)
    is0=np.sum(B2[idx4[(len(idx3[0])/2):]]==False)
    is1=np.sum(B2[idx4[(len(idx3[0])/2):]]==True)

    c_table=np.array([[if0,if1],[is0,is1]])
    Corr[ii]=sst.chi2_contingency(c_table)[1]
    P[ii]=sst.ttest_ind_from_stats(nobs_acc[ii,0],np.std(A[idx2]),len(idx2[0]),
                                   nobs_acc[ii,1],np.std(A[idx1]),len(idx1[0]))[1]
    
    P2[ii]=sst.ttest_ind(np.log(A[idx2]),np.log(A[idx1]))[1]



nobs_acc_all= np.zeros(2)
nobs_acc_err_all= np.zeros(2)

nobs_acc_all[1]=np.mean(nobs_acc[:,1])
nobs_acc_all[0]=np.mean(nobs_acc[:,0])
nobs_acc_err_all[1]=sst.sem(nobs_acc[:,1])
nobs_acc_err_all[0]=sst.sem(nobs_acc[:,0])

#### hidden state activities
FR = np.zeros((2,len(coh_all),100))
for iiii in xrange(2):
    for iii in xrange(len(coh_all)):
        for ii in xrange(100):
            idx = np.where((A1 == ii+1) & (C1 == coh_all[iii]) & (T1 == iiii))
            FR[iiii,iii,ii] = np.mean(h1[idx,0])


In [ ]:
#matplotlib.rcParams.update({'font.size': 8})
from scipy.optimize import curve_fit
import scipy as sy
d = 100-coh_all*100
d2= np.linspace(0,100,100)

aa=[]
for ii in xrange(len(coh_all)):
    aa.append('{0:.1f}'.format(coh_all[ii]*100).rstrip('0').rstrip('.'))
    
# psychometric function
def pf(x, alpha, beta):
    return 1. / (1 + np.exp( -(x-alpha)/beta ))

par0 = sy.array([0., 1])
par, mcov = curve_fit(pf, d, acc1, par0)


p=np.polyfit(np.log(100-coh_all*100),num_obs,1)
x_fit=np.log(100-coh_all*100)
y_fit=x_fit*p[0]+p[1]



#plt.figure(figsize=(8,2.7))
plt.figure(figsize=(5.2,5))

plt.subplot(221)
#plt.semilogx(100-coh_all[[0,2]]*100,acc1[[0,2]],'ro')
plt.semilogx(100-coh_all[[0,1,2,3,4,5]]*100,acc1[[0,1,2,3,4,5]],'ko')
plt.semilogx(d2, pf(d2, par[0], par[1]),'k')
plt.xlim(1,100)
plt.ylim(0.5,1)
plt.xlabel('Signal strength (100 - % noise)')
plt.ylabel('Accuracy')

plt.subplot(222)
#plt.semilogx(100-coh_all[[0,2]]*100,num_obs[[0,2]],'ro')
plt.semilogx(100-coh_all[[0,1,2,3,4,5]]*100,num_obs[[0,1,2,3,4,5]],'ko')
plt.semilogx(100-coh_all*100,y_fit,'k')
plt.xlim(1,100)
plt.xlabel('Signal strength (100 - % noise)')
plt.ylabel('Mean number of observations')

plt.subplot(223)
for ii in xrange(6):
    if np.isin(ii,np.arange(6)):
        plt.plot(np.arange(1,31,stepsize),bin_obs[ii,:30/stepsize].T,'-')
    else:
        bbb=plt.plot(np.arange(1,31,stepsize),bin_obs[ii,:30/stepsize].T,'--')
plt.xlabel('Time')
plt.ylabel('Trial length count')
    
leg = plt.legend(aa,fontsize=7.5)
leg.set_title('Noise')

plt.subplot(224)
bin_sz = 7
lentr = 98
acc[acc==0]=np.nan
#plt.plot(acc.T)
accres = acc[:,:lentr].reshape(6,-1,bin_sz)
naccres = nacc[:,:lentr].reshape(6,-1,bin_sz)
accall=np.nansum(accres,2)/np.nansum(naccres,2)
accall[np.nansum(naccres,2)<50]=np.nan
plt.plot(np.arange(1,lentr,bin_sz),accall.T)
plt.xlim([0,30])
plt.xlabel('Time')
plt.ylabel('Accuracy')

leg = plt.legend(aa,fontsize=7.5,loc='lower right')
leg.set_title('Noise')

plt.tight_layout()

plt.show()

###
plt.figure(figsize=(5.2,2.6))
plt.subplot(121)
for ii in xrange(6):
    if np.isin(ii,np.arange(6)):
        print(stepsize)
        plt.plot(np.arange(1,31,stepsize),bin_obs_co[ii,:30/stepsize].T/np.nansum(bin_obs_co[ii,:30/stepsize]),'-')
    else:
        bbb=plt.plot(np.arange(1,31,stepsize),bin_obs[ii,:30/stepsize].T,'--')
plt.xlabel('Time')
plt.ylabel('Trial length count')
    
leg = plt.legend(aa,fontsize=7.5)
leg.set_title('Noise')

plt.subplot(122)
for ii in xrange(6):
    if np.isin(ii,np.arange(6)):
        print(stepsize)
        plt.plot(np.arange(1,31,stepsize),bin_obs_uc[ii,:30/stepsize].T/np.nansum(bin_obs_uc[ii,:30/stepsize]),'-')
    else:
        bbb=plt.plot(np.arange(1,31,stepsize),bin_obs[ii,:30/stepsize].T,'--')
plt.xlabel('Time')
plt.ylabel('Trial length count')
    
leg = plt.legend(aa,fontsize=7.5)
leg.set_title('Noise')
plt.tight_layout()

###
plt.figure(figsize=(8,2.7))
plt.subplot(131)
plt.plot(coh_all*100,acc1,'ko-')

plt.xlim(1,100)
plt.ylim(0.5,1)
plt.xlabel('Noise')
plt.ylabel('Accuracy')

plt.subplot(132)
plt.plot(coh_all*100,num_obs,'ko-')
plt.xlim(1,100)
plt.xlabel('Noise')
plt.ylabel('Number of observations')

plt.subplot(133)
plt.plot(np.arange(1,31,stepsize),bin_obs[:,:30/stepsize].T,'-')
plt.xlabel('Time')
plt.ylabel('Trial length count')
    
leg = plt.legend(aa,fontsize=7.5)
leg.set_title('Noise')
plt.tight_layout()
plt.show()

###

idx1 = list(np.where((C2 == coh[2]) & (B==1)))
idx2 = list(np.where((C2 == coh[2]) & (B!=1)))

X=np.concatenate((A[idx1],A[idx2])).reshape(-1,1)
y=np.concatenate((B[idx1],B[idx2]))
from sklearn import linear_model
X_test = np.linspace(1, 30, 100)
clf = linear_model.LogisticRegression(C=1e5)
clf.fit(X, y)
def model(x):
    return 1 / (1 + np.exp(-x))
loss = model(X_test * clf.coef_ + clf.intercept_).ravel()
# and plot the result
plt.figure(figsize=(8, 5))
plt.clf()

idx=230
for ii in xrange(len(coh_all)):
    idx=idx+1
    plt.subplot(idx)
    plt.errorbar([0,1],nobs_acc[ii].T, yerr=nobs_acc_err[ii], color='black', capsize=4)
    plt.xlim((-0.5,1.5))
    plt.margins(y=0.5)
    plt.xticks([0,1],('Incorrect','Correct'))
    plt.title('N = '+ str(coh_all[ii]))
    #plt.xlabel('Classification')

    plt.ylabel('Trial length')

plt.xlim((-0.5,1.5))
plt.tight_layout()
plt.show()

plt.figure(figsize=(7,2.3))
plt.subplot(131)
plt.plot(coh_all*100,acc1,'o-')
plt.ylim(0,1)
plt.xlabel('Noise percentage')
plt.ylabel('Accuracy')

plt.subplot(132)
plt.plot(coh_all*100,num_obs-3,'o-')
plt.xlabel('Noise percentage')
plt.ylabel('Number of observations')

plt.subplot(133)
plt.plot(np.arange(1,41,stepsize),bin_obs[:,:40/stepsize].T,'-')
plt.xlabel('Time')
plt.ylabel('Trial length count')


plt.tight_layout()
plt.show()

plt.figure(figsize=(7,3))
plt.subplot(121)
plt.errorbar([0,1],nobs_acc_all.T, yerr=nobs_acc_err_all, color='black', capsize=4)
plt.show()

plt.figure(figsize=(7,3))
plt.bar(np.arange(12), acc_nobs.reshape((12)))
plt.ylim((0.8,1))
plt.show()


In [ ]:
from sklearn.cluster import KMeans
dist= np.zeros(10)
for ii in xrange(10):
    kmeans = KMeans(n_clusters=ii+1, random_state=0).fit(h1.T)
    print(ii)
    dist[ii] = kmeans.inertia_


In [ ]:
cls=4
#aa=str(coh_all*100)[1:-1].split()[:]
#aa='{0:.2f}'.format(aa)
#aa=['N = ' + x for x in aa]
aa=[]
for ii in xrange(len(coh_all)):
    aa.append('{0:.1f}'.format(coh_all[ii]*100).rstrip('0').rstrip('.'))

FR = np.zeros((2,len(coh_all),50))
FR[:] = np.nan
FRH = np.zeros((2,len(coh_all),50))
FRH[:] = np.nan
FRH2 = np.zeros((cls,2,len(coh_all),50))
FRH2[:] = np.nan
hm=[]

from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=cls, random_state=0).fit(h1.T)
for ii in xrange(cls):
    hm.append(np.mean(h1[:,kmeans.labels_==ii],axis=1))
    
for iiii in xrange(2):
    for iii in xrange(len(coh_all)):
        for ii in xrange(50):
            idx = np.where((A1 == ii+1) & (C1 == coh_all[iii]) & (T1 == iiii))
            if len(idx[0])>20:
                FR[iiii,iii,ii] = np.mean(p2[idx])
                FRH[iiii,iii,ii] = np.mean(p3[idx])
                for cl in xrange(cls):
                    FRH2[cl,iiii,iii,ii] = np.mean(hm[cl][idx])

            else:
                FR[iiii,iii,ii] = None
                FRH[iiii,iii,ii] = None
                FRH2[cl,iiii,iii,ii] = None

idx = np.where((A1 == 1) & (C1 == coh_all[5]) & (T1 == 0))


plt.figure()
plt.clf()
plt.plot(FR[0,:,:40].T,'-')
plt.xlabel('Time')
plt.ylabel('FR')
plt.legend(coh_all)
plt.ylim((0,1))
plt.show()

plt.figure()
plt.clf()
plt.plot(FRH[0,:,:40].T,'-')
plt.xlabel('Time')
plt.ylabel('FR')
plt.legend(coh_all)
plt.show()

matplotlib.rcParams.update({'font.size': 10})
plt.figure(figsize=(8,4.9))

cls_order=[0,3,2,1]
print(cls)
plt.clf()
for iii in xrange(2):
    for ii in xrange(cls):

        plt.subplot(2,cls,1+ii+4*iii)
        plt.plot(FRH2[cls_order[ii],iii,:,:40].T,'-')
        plt.xlabel('Time')
        if 1+ii+4*iii==1 or 1+ii+4*iii==5:
            
            plt.ylabel('Resp. stim '+str(iii+1))
            
        if iii==0:
            plt.title('Cluster '+str(ii+1))
        plt.yticks([], [])

        plt.tight_layout()
        plt.ylim((0,1))
        plt.xlim((0,41))
        if ii==2 and iii==0:
            plt.legend(aa,fontsize=8)
plt.show()


###
plt.figure(figsize=(8,2.7))

cls_order=[0,3,2,1]
print(cls)
plt.clf()
for ii in xrange(cls):

    plt.subplot(1,cls,1+ii)
    plt.plot(FRH2[cls_order[ii],0,:,:40].T,'-')
    plt.gca().set_color_cycle(None)
    plt.plot(FRH2[cls_order[ii],1,:,:40].T,'--')

    plt.xlabel('Time')
    if 1+ii==1:
        plt.ylabel('Resp. stim '+str(iii+1))


    plt.title('Cluster '+str(ii+1))
    plt.yticks([], [])

    plt.tight_layout()
    plt.ylim((0,1))
    plt.xlim((0,41))
    if ii==2:
        leg = plt.legend(aa,fontsize=8)
        leg.set_title('Noise')
plt.show()
###

matplotlib.rcParams.update({'font.size': 10})
plt.figure(figsize=(8,2.7))
plt.subplot(131)
plt.plot(FR[1,:,:40].T,'-')
plt.xlabel('Time')
plt.ylabel('P(match)')
plt.ylim((0,1))

plt.subplot(132)
plt.plot(FR[0,:,:40].T,'-')
plt.xlabel('Time')
plt.ylabel('P(non-match)')
plt.ylim((0,1))
leg = plt.legend(aa,fontsize=8)
leg.set_title('Noise')

plt.subplot(133)
plt.plot(FRH[0,:,:40].T,'-')
plt.xlabel('Time')
plt.ylabel('P(observation)')
plt.ylim((0,1))

plt.tight_layout()
plt.show()





In [ ]:
res=150

fname = 'result/model_mnist25_fl'+str(fix_length)+'_l' + str(trial_length) + '_g' + str(gamma).replace('.', '') + '_r' + str(rwrds[0]).replace('.', '')

data=np.load(fname + '_log.npz')
print(data.items())
crwrd=data['arr_0']
r1=np.diff(crwrd)
r1=r1[r1!=0]
r1_last=len(a1)-np.mod(len(a1),res)

print(len(crwrd[0::res]))
rwrd_rate=crwrd[res::res]-crwrd[0:-res:res]
a1=data['arr_1']
NaN_val = [a1==None][0]
a1[a1==None]=0.5

acc_trial=np.nanmean(1.0*np.reshape(a1[:r1_last],(-1,res)),axis=1)
resp_rat = 1-np.nanmean(1.0*np.reshape(NaN_val[:r1_last],(-1,res)),axis=1)

#actions
action = data['arr_2']
idx = np.where(action<2)
T_len = np.diff(idx[0])
idx2 = [T_len>2]
idx3 = np.insert(idx2,np.where(T_len>=50)[0],False)
idx4 = np.insert(idx3,np.where(T_len>=100)[0],False)

print(len(a1))

plt.figure(figsize=(8,2.6))
plt.clf()
plt.subplot(131)
plt.plot(crwrd,'k')
plt.xlabel('Trial')
plt.ylabel('Cumulative reward')

plt.subplot(133)
trials = np.arange(0,len(acc_trial))*r1_last/len(acc_trial)
plt.plot(trials,acc_trial,'k')
plt.xlabel('Trial')
plt.ylabel('Accuracy')
plt.ylim([0,1])

plt.subplot(132)
trials = np.arange(0,len(acc_trial))*r1_last/len(acc_trial)
plt.plot(trials,resp_rat,'k')
plt.xlabel('Trial')
plt.ylabel('% responded in time')
plt.ylim([0,1])


plt.tight_layout()
plt.show()


jkl = np.arange(10)


In [ ]:
# Speed accuracy trade-off
aa=str(coh_all)[1:-1].split()[:]
aa=['N = ' + x for x in aa]

from cycler import cycler
acc_sa=np.empty((3,6))
nobs_sa=np.empty((3,6))

acc_sa[2]= np.asarray([0.99318569,0.99353796,0.98214286,0.9609375,0.81722054,0.705])
nobs_sa[2]=np.asarray([4.70357751,4.60581583,5.07142857,5.5515625,6.0407855,6.18166667])

acc_sa[1]=np.asarray([0.99007444,0.98726115,0.97811816,0.94209354,0.80694143,0.72463768])
nobs_sa[1]=np.asarray([6.0942928,6.05307856,7.04595186,7.86414254,8.78958785,9.39130435])

acc_sa[0]=np.asarray([0.98477157,0.99137931,0.97101449,0.93859649,0.8893617,0.74369748])
nobs_sa[0]=np.asarray([6.24365482,7.64655172,10.69082126,16.26754386,20.88510638,23.98739496])

cost=[0, -0.005, -0.01]
matplotlib.rcParams.update({'font.size': 10})
print(coh_all)
plt.figure(20,figsize=(6,5.7))
plt.subplot(221)
plt.xlabel('Noise')

plt.semilogx(100-coh_all*100,acc_sa.T,'o-')

plt.ylim(0.65,1)
ax0 = plt.gca()
#ax0.invert_xaxis()

plt.ylabel('Accuracy')
plt.legend(['Cost = 0','Cost = -0.005','Cost = -0.01'],fontsize=8)


plt.subplot(223)
plt.plot(coh_all*100,nobs_sa.T,'o-')
ax0 = plt.gca()
#ax0.invert_xaxis()
plt.xlabel('Noise')
plt.ylabel('Number of observations')

plt.subplot(222)
plt.plot(cost,acc_sa,'o-')
ax1 = plt.gca()
ax1.invert_xaxis()
plt.xlabel('Cost of observation')
plt.ylabel('Accuracy')
plt.xticks(cost)

plt.subplot(224)
plt.plot(cost,nobs_sa,'o-')
ax2 = plt.gca()
ax2.invert_xaxis()

plt.xlabel('Cost of observation')
plt.ylabel('Number of time-steps')
plt.xticks(cost)
leg = plt.legend(aa,fontsize=8)
leg.set_title('Noise')

plt.tight_layout()
plt.show()


In [ ]:
n_hidden = 100
tsteps = 20
# get variables
Y1 = monitor['action']
h1 = monitor1['hidden-1']

# calculate trial time steps
A1 = np.zeros(Y1.shape)
temp = 0
for ii in xrange(len(Y1)):
    temp = temp + 1
    A1[ii] = temp
    if Y1[ii] != (data_iter.n_output - 1) or temp == trial_length+1:
        temp = 0


# only focus on those trials at which a decision is being made
idx = np.where((Y1 != data_iter.n_output - 1) & (A1==tsteps))[0]
Y = Y1[idx]

R = np.zeros((len(idx),tsteps,n_hidden))
for ii in xrange(len(idx)):
    R[ii] = h1[(idx[ii]-tsteps+1):(idx[ii]+1)]

A = np.tile(np.expand_dims(val_iter.mean_image_count,2),reps = (1,1,784))
B=1.0*val_iter.mean_image/A
C=B[0,:,:]-B[1,:,:]
D=np.mean(C**2,1)




In [ ]:
C = Y
CP = np.zeros((tsteps,n_hidden))
for t in xrange(tsteps):
    for N in xrange(n_hidden):
        n = np.sum(C==0)
        m = np.sum(C==1)
        k = np.linspace(np.max(R[C==1,t,N]),np.min(R[C==1,t,N]),100)
        ROC = np.zeros((2,100))
        for ii in xrange(100):
            
            NTP = np.sum(R[C==0,t,N]>k[ii])
            NFP = np.sum(R[C==1,t,N]>k[ii])
            ROC[0,ii] = 1.0*NTP/n
            ROC[1,ii] = 1.0*NFP/m
        CP[t,N] = np.sum(np.diff(ROC[1,:])*ROC[0,:-1])
        
A=CP[:,np.mean(CP[3:,:],axis=0)>0.5]


plt.figure()
plt.plot(np.mean(A,axis=1))
plt.xlabel('Time')
plt.ylabel('CP')

plt.figure()
plt.plot(CP)
plt.xlabel('Time')
plt.ylabel('CP')

In [ ]:
plt.figure()
plt.imshow(np.reshape((B[0,10,:]-B[1,10,:]),(28,28)))



In [ ]:
plt.figure(figsize=(7,3))
plt.clf()
plt.subplot(121)
plt.plot(np.mean(A,axis=1))
plt.xlabel('Time')
plt.ylabel('CP')
plt.ylim([0,1])


sp2 = plt.subplot(122)
plt.plot(D)
plt.xlabel('Time')
plt.ylabel('PK')
sp2.axes.set_yticklabels([])
plt.tight_layout()

plt.show()
